In [ ]:
from dotenv import load_dotenv

load_dotenv()

import duckdb
import polars as pl
import plotly.express as px
from src.settings import settings

In [ ]:
%load_ext sql
conn = duckdb.connect(settings.DUCKDB_DATABASE)
conn.sql("SET search_path TO public;")

In [ ]:
df = conn.sql("""
SELECT
    CONCAT(
        CAST(date_part('year', ca.date) AS VARCHAR),
        'Q',
        CAST(date_part('quarter', ca.date) AS VARCHAR)
    ) AS quarter,
    AVG(ca.tave) AS tave,
    AVG(ca.tmin) AS tmin,
    AVG(ca.tmax) AS tmax,
    AVG(heat_index) AS heat_index,
    AVG(pr) AS pr,
    AVG(wind_speed) AS wind_speed,
    AVG(rh) AS rh,
    AVG(solar_rad) AS solar_rad,
    AVG(uv_rad) AS uv_rad
FROM cchain__climate_atmosphere ca
LEFT JOIN cchain__location l ON ca.adm4_pcode = l.adm4_pcode
WHERE l.adm1_en = 'Region III'
GROUP BY quarter
ORDER BY quarter
""").pl()
df

In [ ]:
px.line(
    df,
    x='quarter',
    y='pr',
    title='Quarterly Precipitation in Region III (2003-2022)',
)